In [1]:
import socket
import string 
import random
import pandas as pd
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import numpy as np

In [2]:
HOST = '127.0.0.1'
PORT = 55555

In [3]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  #stream eh TCP
s.connect((HOST, PORT))

In [4]:
#Faz a criptografia
def encryption(data):
    coded = ''
    hash = len(data)-1
    i = hash
    while i>= 0:
        coded = coded + random.choice(string.ascii_letters)
        coded = coded + data[i]
        coded = coded + random.choice(string.ascii_letters+string.digits)    
        i -= 1
    return coded

In [5]:
#Faz a decriptografia
def decryption(data):
    decoded = ''
    hash = len(data)-1
    i = hash 
    i -= 1
    while i >= 0:
        decoded = decoded + data[i]
        i -= 3
    return decoded

In [6]:
# codifica a string criptografada de ASCII para 8B6T
def encode(data):
    encoded = ''
    table = pd.read_csv('8b6tTABLE.csv')
    table.fillna(' ', inplace = True)
    for i in range(len(data)):
        for j in range(len(table)):
            if(table['Char'].iloc[j] == data[i]):    
                encoded = encoded + flip(table['6T'].iloc[j])
    
    return encoded

In [7]:
# decodifica de 8b6T para ASCII
def decode(data):
    decoded = ''
    table = pd.read_csv('8b6tTABLE.csv')
    table.fillna(' ', inplace = True)
    i = 0
    k = 6
    for k in range(len(data)):
        for j in range(len(table)):
            if(table['6T'].iloc[j] == flip(data[0:6])):
                decoded = decoded + table['Char'].iloc[j]
                data = data[6:]
    return decoded

In [8]:
# muda os valores do sinal se tiver dois positivos seguidos
def flip(data):
    flipped = ''
    counter = 0
    for i in range(len(data)):
        if data[i] == '+':
            flipped = flipped + '-'
            counter += 1
        elif data[i] == '-':
            flipped = flipped + '+'
            counter -= 1
        else:
            flipped = flipped + '0'
    if counter >= 1:
        return flipped
    if counter <= -1:
        return flipped
    else: return data

In [9]:
def homeOne():
    sg.theme = 'Black'
    #layout
    layout=[
        [sg.Text('Mensagem:'), sg.Input()],
        [sg.Button('Enviar')],
        #[sg.Output()]
    ]

    #Janela
    janela=sg.Window("Escreva uma mensagem",layout).Finalize()
    
    
    #ColetaDados
    event, values= janela.Read()
    
    
    janela.close()
    return values[0]
             

In [10]:
def paraBinario(data):
    #myString = ''
    #ba = bytearray(data)
    #ba[0]
    return (''.join('{:08b}'.format(ord(c)) for c in data))
    #return (bin(ba[0]))


In [11]:
def grafico(data):
    #print(f'teste data',data)
    #dados = ''
    #print(f'teste dados',dados)
    #barWidth=0.25

    plt.figure()#figsize=(10,5))

    r1= np.arange(len(data))

    plt.bar(r1, data , color='#6A5ACD')#, width=barWidth)

    plt.show()

In [12]:
def homeTwo(data, aux3, aux2):
    sg.theme = 'Black'
    #layout
    layout=[
        [sg.Output(size=(50,30))],
        [sg.Button('Sair')]
    ]
    
    #Janela
    janela=sg.Window("Mensagens",layout).Finalize()
    print(f'Mensagem:', aux3)
    print(f'Mensagem binario:', paraBinario(aux3))
    #aux4 = encryption(aux3)
    aux3 = encryption(aux3)
    print(f'Mensagem criptografada: ', aux3)
    aux3 = encode(aux3)
    print(f'Mensagem 8b6t:', aux2)
    #print(f'Mensagem 8b6t:', encode(aux4))
    
    #grafico(data)
    
    while True:
        button, values =janela.Read()
        if button == 'Sair':
            grafico(data)
        else:
            janela.close()
    janela.close()

In [13]:
#main
mensagem = homeOne()
aux1 = mensagem
mensagem = encryption(mensagem)
print(f'testando clipto: ', mensagem)
mensagem = encode(mensagem)
aux2=mensagem
print(f'testando encode: ', mensagem)
s.sendall(str.encode(mensagem))

data = s.recv(1024) 

tela=homeTwo(data, aux1, aux2)

#grafico(aux2)

#print(f'Mensagem ecoada:', data.decode())

testando clipto:  po5hixQuvyyCQtfYrBSeIbw7Eqp
testando encode:  -0+0000-+0000--++000+--000-000++---0+-00-+-++--0++-0--++0---++0---+-0+--+-00-++-+--000+0--+-00+-+-+0--+00+--0+-0-++00--00++-0-0+00--++--0---0+0+-+0-00-++-0--0+000


ValueError: shape mismatch: objects cannot be broadcast to a single shape